In [ ]:

splits = 10
########
tx = range(10)
ty = [0] * 5 + [1] * 5

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import datasets

#kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
shufflesplit = StratifiedShuffleSplit(n_splits=splits, random_state=42, test_size=4)

#print("KFold")
#for train_index, test_index in kfold.split(tx, ty):
   # print("TRAIN:", train_index, "TEST:", test_index)

print("Shuffle Split")
for train_index, test_index in shufflesplit.split(tx, ty):
    print("TRAIN:", train_index, "TEST:", test_index)
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
import time

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#df_test=pd.read_csv('test.csv')
#df_test.head()

In [ ]:
df_train=pd.read_csv('train.csv')
df_train.head()

In [ ]:
print('Any Nan Value in Train dataset',df_train.isnull().sum().sum())

In [ ]:
df_train['target'].value_counts()

In [ ]:
print('For 1 is',160/250)
print('For 0 is',90/250)

In [ ]:
### Plot when target is equals to 1
plt.plot(df_train[df_train.target==1].mean(axis=1))

In [ ]:
### Plot when target is equals to 0
plt.plot(df_train[df_train.target==0].mean(axis=1))

In [ ]:
#myarray = np.asarray(df_train[df_train.target==1].mean(axis=1))
#sns.distplot(myarray)
################################################################################################
#myarray = np.asarray(df_train[df_train.target==0].mean(axis=1))
#sns.distplot(myarray)

In [ ]:
corr=df_train.drop(['id','target'],axis=1).corr()
plt.figure(figsize=(25,25))
sns.heatmap(corr)

In [ ]:
df_features=df_train.drop(['id','target'],axis=1)
cols = df_features.columns
# Calculates pearson co-efficient for all combinations
data_corr = df_features.corr()
# Set the threshold to select only highly correlated attributes
threshold = 0.25

# List of pairs along with correlation above threshold
corr_list = []

size=len(df_features.columns)
for i in range(0,size):
    for j in range(i+1,size):
        if (data_corr.iloc[i,j] >= threshold and data_corr.iloc[i,j] < 1) or (data_corr.iloc[i,j] < 0 and data_corr.iloc[i,j] <= -threshold):
            corr_list.append([data_corr.iloc[i,j],i,j])
            
#Sort to show higher ones first            
s_corr_list = sorted(corr_list,key=lambda x: -abs(x[0]))

#Print correlations and column names
for v,i,j in s_corr_list:
    print ("%s and %s = %.2f" % (cols[i],cols[j],v))

In [ ]:
plt.figure(figsize=(45,30))
for i, col in enumerate(list(df_train.columns)):
    plt.subplot(17, 50, i + 2)
    plt.hist(df_train[col])
    plt.title(col)

In [ ]:
#plt.plot(corr.var(axis=0))

#### Train-Test Split--Stratified Shuffle split
#### Taking Test size large as 80% and train size 20%

In [ ]:
df_feature=df_train.drop(['id','target'],axis=1)
X=np.array(df_features)
y=np.array(df_train['target'])
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
grid=[{"C":np.logspace(-3,1,7)}]
print('grid',grid)

In [ ]:
%%time
C=np.array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01])
print('C',C)
## n_splits ## Number of Bootstrap Splits
## n ## Number of KFold Splits
def Accuracy(X,y,n,n_splits):
    l_accu_score=list()
    key_c=list() ### 
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=123)
    
    for subset_index, test_index in sss.split(X, y):
        X_train_ss=X[subset_index]
        Y_train_ss=y[subset_index]
       # X_test_ss=X[test_index]
       # Y_test_ss=y[test_index]
        
        for c in C:
            #print('C',c)
            key_c.append(c)
            kf = KFold(n_splits =n)
            for train_index, test_index in kf.split(X_train_ss):
                X_train, X_test = X_train_ss[train_index], X_train_ss[test_index]
                y_train, y_test = Y_train_ss[train_index], Y_train_ss[test_index]
                
                logmodel = LogisticRegression(C=c,penalty='l1',max_iter = 100000).fit(X_train,y_train)
                ypred = logmodel.predict(X_test)
                
                l_accu_score.append(accuracy_score(y_test,ypred))
                new_list = [l_accu_score[i:i+n] for i in range(0, len(l_accu_score), n)]
                
                #print ('Accuracy Score :',(accuracy_score(y_test,ypred)))
    return new_list,key_c
new_list,key_c=Accuracy(X,y,10,2000)

In [ ]:
def best_c(key_c,new_list):
    df=pd.concat([pd.DataFrame(key_c,columns = list("C")),pd.DataFrame(new_list).mean(axis=1)],axis=1)
    return df.groupby('C').mean()
best_c(key_c,new_list)

In [ ]:
import random
from functools import reduce



new_coef=list()
accuracy=list()
for i in range(1,1000):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=random.randint(1,1000))
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    for train_index, test_index in sss.split(X, y):
        X_train=X[train_index]
        Y_train=y[train_index]
        x_test=X[test_index]
        y_test=y[test_index]
        
        scaler = StandardScaler()
        
        
        
        logmodel=LogisticRegression(C= .100000,penalty='l1',max_iter = 100000).fit(X_train,Y_train)
        ypred = logmodel.predict(x_test)
        print ('Accuracy Score :',(accuracy_score(y_test,ypred)))
        accuracy.append(accuracy_score(y_test,ypred))
        
        coef=pd.Series(reduce(lambda x,y :x+y ,logmodel.coef_),df_features.columns)
        coefficents=list(coef[coef!=0].index)
        new_coef.append(coefficents)
        print(coefficents)
        
        #new_coef = [coefficents[i:i+len(coefficents)] for i in range(0, len(coefficents),len(coefficents))]
        
        
    

In [ ]:
len(new_coef)

In [ ]:
dict_coeff={}
for wds in (range(len(new_coef))):
    for w in new_coef[wds]:
        if w in dict_coeff:
            dict_coeff[w]=dict_coeff[w]+1
        else:
            dict_coeff[w]=1

In [ ]:
print('Maximum',max(accuracy))
print('Minnimum',min(accuracy))
print('Minnimum',reduce(lambda x, y: x + y, accuracy) / len(accuracy))

In [ ]:
regular_coeff=([k for (k,v) in dict_coeff.items() if v >500])
print('Length of list',len(regular_coeff))
#regular_coeff

In [ ]:
## Appending id column
regular_coeff.append('id')
regular_coeff.append('target')
df_train2=pd.read_csv('train.csv',usecols=regular_coeff)
df_train2.head()

In [ ]:
df_feature=df_train2.drop(['id','target'],axis=1)
X=np.array(df_feature)
y=np.array(df_train2['target'])
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X.shape

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=123)

accuracy_relaxed=list()
for i in range(1,1000):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=random.randint(1,1000))
    for train_index, test_index in sss.split(X, y):
        X_train=X[train_index]
        Y_train=y[train_index]
        x_test=X[test_index]
        y_test=y[test_index]
        
        #print('length of x_test set',(np.array(x_test)).shape)
        #print('length of X_train set',(np.array(X_train)).shape)
        #print('length of Y_train set',(np.array(Y_train)).shape)
        #print('length of y_test set',(np.array(y_test)).shape)
       
        
        
        
        logmodel_relaxed=LogisticRegression(C= .1,penalty='l1',max_iter = 100000).fit(X_train,Y_train)
        ypred = logmodel_relaxed.predict(x_test)
        print ('Accuracy Score :',accuracy_score(y_test,ypred))
        accuracy_relaxed.append(accuracy_score(y_test,ypred))
        
            

In [ ]:
print('Maximum',max(accuracy_relaxed))
print('Minnimum',min(accuracy_relaxed))
print('Minnimum',reduce(lambda x, y: x + y, accuracy_relaxed) / len(accuracy_relaxed))

In [ ]:
Maximum 0.92
Minnimum 0.58
Minnimum 0.771771771771770

In [ ]:
## Appending id column
regular_coeff.append('id')

In [ ]:
regular_coeff

In [ ]:
regular_coeff.remove('target')

In [ ]:
df_test=pd.read_csv('test.csv',usecols=regular_coeff)
df_test.head()

In [ ]:
df_test_features=df_test.drop(['id'],axis=1)
df_test_features.head()

In [ ]:
logmodel_relaxed.coef_

In [ ]:
X_test=np.array(df_test_features)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

In [ ]:
ypred_test = logmodel_relaxed.predict(X_test)

In [ ]:
final=pd.concat([df_test['id'],pd.DataFrame(ypred_test)],axis=1)
final

In [ ]:
final.to_csv('submission.csv',index=False)